In [1]:
# Cell 1: 패키지 설치 및 임포트
# !pip install langchain langchain-openai python-dotenv openai requests

import os
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI
from langchain.schema import (
    SystemMessage,
    HumanMessage,
)
from langchain.callbacks import StreamingStdOutCallbackHandler
import openai
import requests
import json

In [2]:
# Cell 2: Azure OpenAI 설정 (.env 파일 사용)
# .env 파일에서 환경 변수 로드
load_dotenv()

# Azure OpenAI 환경 변수에서 값 가져오기
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
api_version = os.getenv("AZURE_OPENAI_API_VERSION")
deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

# 설정 값 검증
required_vars = {
    "AZURE_OPENAI_ENDPOINT": azure_endpoint,
    "AZURE_OPENAI_API_KEY": api_key,
    "AZURE_OPENAI_API_VERSION": api_version,
    "AZURE_OPENAI_DEPLOYMENT_NAME": deployment_name
}

missing_vars = [name for name, value in required_vars.items() if not value]

if missing_vars:
    print("❌ 환경 변수가 누락되었습니다:")
    for var in missing_vars:
        print(f"  - {var}")
    print("\n.env 파일을 확인해주세요.")
else:
    print("✅ Azure OpenAI 설정 로드 완료:")
    print(f"Endpoint: {azure_endpoint}")
    print(f"Deployment: {deployment_name}")
    print(f"API Version: {api_version}")
    print(f"API Key: {'***' + api_key[-4:] if api_key else 'Not Found'}")

✅ Azure OpenAI 설정 로드 완료:
Endpoint: https://hspar-m7k2pfor-swedencentral.openai.azure.com/
Deployment: gpt-4o-mini
API Version: 2025-01-01-preview
API Key: ***RdJz


In [3]:
# Cell 3: LangChain Azure OpenAI 초기화
try:
    llm = AzureChatOpenAI(
        azure_endpoint=azure_endpoint,
        api_version=api_version,
        azure_deployment=deployment_name,
        api_key=api_key,
        temperature=0.7,
        max_tokens=2048
    )
    
    print("✅ LangChain Azure OpenAI 모델이 초기화되었습니다.")
    
except Exception as e:
    print(f"❌ LangChain 초기화 실패: {e}")
    llm = None

✅ LangChain Azure OpenAI 모델이 초기화되었습니다.


In [4]:
# Cell 4: 기본 테스트 메시지 전송
def test_basic_chat():
    """
    기본 채팅 테스트
    """
    if llm is None:
        print("❌ 모델이 초기화되지 않았습니다.")
        return False
    
    messages = [
        SystemMessage(content="You are a helpful AI assistant."),
        HumanMessage(content="Hello! Please introduce yourself and tell me what you can help with.")
    ]
    
    try:
        print("메시지 전송 중...")
        response = llm.invoke(messages)
        print("\n✅ 응답 성공:")
        print(response.content)
        return True
        
    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        return False

# 기본 테스트 실행
success = test_basic_chat()

메시지 전송 중...

✅ 응답 성공:
Hello! I’m an AI assistant here to help you with a wide range of topics. I can provide information, answer questions, assist with problem-solving, and offer suggestions on various subjects such as technology, science, history, literature, and more. Whether you need help with research, writing, learning new concepts, or just have general inquiries, feel free to ask! How can I assist you today?


In [5]:
# Cell 5: OpenAI 클라이언트 직접 사용 테스트
def test_openai_client():
    """
    OpenAI 클라이언트 직접 사용 테스트
    """
    try:
        # OpenAI 클라이언트 초기화
        client = openai.AzureOpenAI(
            azure_endpoint=azure_endpoint,
            api_key=api_key,
            api_version=api_version
        )
        
        print("OpenAI 클라이언트로 테스트 중...")
        
        response = client.chat.completions.create(
            model=deployment_name,
            messages=[
                {"role": "system", "content": "You are a helpful AI assistant."},
                {"role": "user", "content": "What are the main features of Azure OpenAI?"}
            ],
            max_tokens=1000,
            temperature=0.7
        )
        
        print("\n✅ OpenAI 클라이언트 응답 성공:")
        print(response.choices[0].message.content)
        return True
        
    except Exception as e:
        print(f"❌ OpenAI 클라이언트 오류: {e}")
        return False

# OpenAI 클라이언트 테스트 실행
client_success = test_openai_client()

OpenAI 클라이언트로 테스트 중...

✅ OpenAI 클라이언트 응답 성공:
Azure OpenAI Service provides a range of powerful features that leverage OpenAI's advanced models, allowing businesses and developers to integrate AI capabilities into their applications. Here are the main features of Azure OpenAI:

1. **Access to OpenAI Models**: Users can access various state-of-the-art models, including GPT (Generative Pre-trained Transformer) for natural language processing tasks, Codex for code generation, and DALL-E for image generation.

2. **Scalability**: Azure OpenAI is built on Microsoft Azure’s cloud infrastructure, allowing users to scale their applications seamlessly according to demand.

3. **Security and Compliance**: Azure OpenAI ensures enterprise-grade security, including data encryption, compliance with regulations (such as GDPR), and governance features to manage data access and usage.

4. **Customizability**: Users can fine-tune models for specific use cases or industries, enabling the generation of ta

In [6]:
# Cell 6: 대화형 함수 정의
def chat_with_azure_openai(user_message, system_message="You are a helpful AI assistant."):
    """
    Azure OpenAI와 대화하는 함수
    """
    if llm is None:
        return "❌ 모델이 초기화되지 않았습니다."
    
    messages = [
        SystemMessage(content=system_message),
        HumanMessage(content=user_message)
    ]
    
    try:
        response = llm.invoke(messages)
        return response.content
    except Exception as e:
        return f"❌ 오류: {e}"

# 다양한 질문 테스트
test_questions = [
    "Azure OpenAI와 일반 OpenAI의 차이점은 무엇인가요?",
    "Python으로 간단한 웹 크롤러를 만드는 방법을 알려주세요",
    "머신러닝과 딥러닝의 차이점을 설명해주세요",
    "서울의 관광명소 5곳을 추천해주세요"
]

print("=== 다양한 질문 테스트 ===")
for i, question in enumerate(test_questions, 1):
    print(f"\n🤔 질문 {i}: {question}")
    answer = chat_with_azure_openai(question)
    print(f"🤖 답변: {answer}")
    print("-" * 80)

=== 다양한 질문 테스트 ===

🤔 질문 1: Azure OpenAI와 일반 OpenAI의 차이점은 무엇인가요?
🤖 답변: Azure OpenAI와 일반 OpenAI의 차이점은 주로 서비스의 배포 방식, 접근 방법, 그리고 추가적으로 제공되는 기능들에 있습니다. 아래에 주요 차이점을 정리해 드리겠습니다.

1. **플랫폼 및 배포**:
   - **Azure OpenAI**: Microsoft Azure 클라우드 플랫폼에서 제공되는 OpenAI 모델입니다. Azure의 인프라를 통해 배포되며, Azure의 다양한 서비스와 통합하여 사용할 수 있습니다.
   - **일반 OpenAI**: OpenAI의 공식 웹사이트나 API를 통해 직접 제공되는 서비스입니다. OpenAI의 자체 인프라에서 운영됩니다.

2. **접근 방법 및 인증**:
   - **Azure OpenAI**: Azure 계정을 통해 접근하며, Azure의 인증 및 권한 관리 시스템을 사용합니다. Azure Subscription이 필요합니다.
   - **일반 OpenAI**: OpenAI의 API 키를 통해 직접 접근하며, 별도의 사용자 관리 시스템을 사용합니다.

3. **기능 및 통합**:
   - **Azure OpenAI**: Azure의 AI 서비스, 데이터 저장소, 분석 도구 등과 쉽게 통합할 수 있는 기능이 제공됩니다. Azure의 보안 및 관리 기능을 활용할 수 있습니다.
   - **일반 OpenAI**: OpenAI에서 제공하는 기본적인 API 기능에 초점을 맞추고 있으며, Azure의 다양한 서비스와의 통합 기능은 제공되지 않습니다.

4. **요금 및 과금 모델**:
   - **Azure OpenAI**: Azure의 요금 정책에 따라 과금되며, 사용량에 따라 요금이 부과됩니다. Azure의 다양한 가격 옵션을 활용할 수 있습니다.
   - **일반 OpenAI**: OpenAI의 자체 가격 정책에 따라 요금이 부과됩니다.

5. **지원 및 SLA**:
   - 

In [7]:
# Cell 7: 스트리밍 응답 테스트
def test_streaming_response():
    """
    스트리밍 응답 테스트
    """
    try:
        # 스트리밍용 모델 초기화
        streaming_llm = AzureChatOpenAI(
            azure_endpoint=azure_endpoint,
            api_version=api_version,
            azure_deployment=deployment_name,
            api_key=api_key,
            temperature=0.7,
            max_tokens=2048,
            streaming=True,
            callbacks=[StreamingStdOutCallbackHandler()]
        )
        
        print("=== 스트리밍 응답 테스트 ===")
        print("질문: 인공지능의 미래에 대해 500자 정도로 설명해주세요.\n")
        print("응답:")
        
        messages = [
            SystemMessage(content="You are a helpful AI assistant."),
            HumanMessage(content="인공지능의 미래에 대해 500자 정도로 설명해주세요.")
        ]
        
        response = streaming_llm.invoke(messages)
        print(f"\n\n✅ 스트리밍 완료!")
        return True
        
    except Exception as e:
        print(f"❌ 스트리밍 오류: {e}")
        return False

# 스트리밍 테스트 실행
streaming_success = test_streaming_response()

=== 스트리밍 응답 테스트 ===
질문: 인공지능의 미래에 대해 500자 정도로 설명해주세요.

응답:
인공지능(AI)의 미래는 여러 측면에서 혁신적인 변화를 가져올 것으로 예상됩니다. 첫째, AI는 산업 전반에 걸쳐 자동화와 효율성을 높이는 데 기여할 것입니다. 제조업, 농업, 물류 등 다양한 분야에서 AI 시스템이 도입되어 생산성을 극대화하고 비용을 절감할 수 있습니다.

둘째, AI는 의료 분야에서도 큰 발전을 이룰 것으로 보입니다. 진단, 치료 계획 수립, 개인 맞춤형 의료 서비스 등에서 AI의 활용이 증가하면서 환자 치료의 질이 향상될 것입니다. 예를 들어, AI는 대량의 데이터를 분석하여 조기 진단을 가능하게 하고, 환자의 유전적 정보를 기반으로 맞춤형 치료를 제공할 수 있습니다.

셋째, 일상생활에서도 AI의 역할이 확대될 것입니다. 스마트 홈 기기, 개인 비서, 자율주행차 등은 우리의 생활을 더욱 편리하고 안전하게 만들어줄 것입니다. 하지만 이러한 발전과 함께 윤리적 문제와 일자리 감소 등의 도전 과제가 동반될 수 있습니다.

결론적으로, AI의 미래는 기술적 혁신과 함께 사회적 책임을 동반해야 하며, 이를 통해 인류의 삶을 개선하는 방향으로 나아가야 할 것입니다. AI가 긍정적인 변화를 이끌어내기 위해서는 기술 발전뿐만 아니라 윤리적 고려와 정책적인 노력이 함께 이루어져야 합니다.

✅ 스트리밍 완료!


In [8]:
# Cell 8: 토큰 사용량 및 비용 추정
def test_with_token_usage():
    """
    토큰 사용량을 포함한 테스트
    """
    try:
        client = openai.AzureOpenAI(
            azure_endpoint=azure_endpoint,
            api_key=api_key,
            api_version=api_version
        )
        
        response = client.chat.completions.create(
            model=deployment_name,
            messages=[
                {"role": "system", "content": "You are a helpful AI assistant."},
                {"role": "user", "content": "머신러닝의 주요 알고리즘 5가지를 설명해주세요."}
            ],
            max_tokens=1500,
            temperature=0.7
        )
        
        print("✅ 응답:")
        print(response.choices[0].message.content)
        
        print(f"\n📊 토큰 사용량:")
        print(f"  - 입력 토큰: {response.usage.prompt_tokens}")
        print(f"  - 출력 토큰: {response.usage.completion_tokens}")
        print(f"  - 총 토큰: {response.usage.total_tokens}")
        
        return True
        
    except Exception as e:
        print(f"❌ 토큰 사용량 테스트 오류: {e}")
        return False

# 토큰 사용량 테스트 실행
token_success = test_with_token_usage()

✅ 응답:
머신러닝의 주요 알고리즘은 다양하지만, 여기서는 대표적인 5가지 알고리즘을 소개하겠습니다.

1. **선형 회귀 (Linear Regression)**:
   - 선형 회귀는 연속적인 값을 예측하는 데 사용되는 알고리즘입니다. 입력 변수와 출력 변수 간의 선형 관계를 모델링합니다. 예를 들어, 주택 가격 예측에 사용될 수 있습니다. 기본적으로, 입력 변수의 가중치를 학습하여 최적의 직선을 찾는 방식으로 작동합니다.

2. **로지스틱 회귀 (Logistic Regression)**:
   - 로지스틱 회귀는 이진 분류 문제에 사용되는 알고리즘입니다. 입력 변수를 기반으로 특정 클래스에 속할 확률을 계산하고, 이 확률을 바탕으로 클래스 레이블을 예측합니다. 결과는 0과 1 사이의 확률로 나타나며, 일반적으로 0.5를 기준으로 클래스가 결정됩니다.

3. **결정 트리 (Decision Tree)**:
   - 결정 트리는 데이터를 분류하거나 회귀하는 데 사용되는 비선형 모델입니다. 데이터를 여러 조건에 따라 분기하여 트리 구조를 형성합니다. 각 노드는 특정 특성에 대한 질문을 나타내며, 리프 노드는 최종 예측 결과를 나타냅니다. 직관적이고 해석하기 쉬운 장점이 있습니다.

4. **서포트 벡터 머신 (Support Vector Machine, SVM)**:
   - SVM은 분류 및 회귀 문제에 사용되는 강력한 알고리즘입니다. 데이터 포인트 간의 경계를 최적화하여 서로 다른 클래스를 분리하는 초평면을 찾습니다. 특히, 고차원 데이터에서 잘 작동하며, 커널 기법을 사용하여 비선형 분류 문제도 처리할 수 있습니다.

5. **신경망 (Neural Networks)**:
   - 신경망은 인간의 뇌 구조를 모방한 알고리즘으로, 복잡한 패턴 인식 및 예측 문제에 매우 효과적입니다. 여러 개의 층(입력층, 은닉층, 출력층)으로 구성되어 있으며, 각 층은 노드(뉴런)로 이루어져 있습니다. 딥러닝이라고 불리는 심층 신경망을 통해 이미지 인식, 자연어 처리 등에서 